ChatGPT Functions

More info in related LinkedIn article @

In [ ]:
!pip install openai -q

In [ ]:
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import requests

In [ ]:
# GPT_MODEL = "gpt-3.5-turbo-0613"
GPT_MODEL = "gpt-4-0613"

In [ ]:
openai.api_key = 'sk-...'

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [ ]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

Function Specification

In [ ]:
functions = [
{
		"name": "extract_speaker_name",
		"description": "Extract speaker first and last name from a given text",
		"parameters": {
			"type": "object",
			"properties": {
				"firstName": {
					"type": "string",
					"description": "The first name of the speaker"
				},
				"lastName": {
					"type": "string",
					"description": "The last name of the speaker"
				}
			},
			"required": ["firstName", "lastName"]
		}
	},
	{
		"name": "extract_company_name",
		"description": "Extract company name from a speaker bio",
		"parameters": {
			"type": "object",
			"properties": {
				"companyName": {
					"type": "string",
					"description": "The company name"
				}
			},
			"required": ["comnpanyName"]
		}
	}
]

In [64]:
bio = """
Stephan Janssen, the orchestrator of Devoxx Belgium, is fervently
invested in conducting research and development (R&D) initiatives.
These efforts have led to innovative features in Devoxx-related
applications, including the CFP web app. In recent years, Stephan
has been delving into state-of-the-art technologies such as large
language models (LLMs), convolutional neural networks (CNNs), and
other artificial intelligence (AI) tools. In his upcoming talk,
he plans to share his practical knowledge and experiences with
the developer community, complete with hands-on demonstrations
and code examples. You can follow him on Twitter: @stephan007.
"""

In [65]:
conversation = Conversation()
conversation.add_message("user", "Extract company name from bio: " + bio)


In [66]:
#with function call
chat_response = chat_completion_request(
    conversation.conversation_history,
    functions = functions
)

chat_response.json()

{'id': 'chatcmpl-7SLeK4UXKhjMbMzXT7XdQQnOBi5tA',
 'object': 'chat.completion',
 'created': 1686990760,
 'model': 'gpt-4-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'extract_company_name',
     'arguments': '{\n  "companyName": "Devoxx Belgium"\n}'}},
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 252, 'completion_tokens': 20, 'total_tokens': 272}}

In [67]:
import json
json.loads(chat_response.json()['choices'][0]['message']['function_call']['arguments'])

{'companyName': 'Devoxx Belgium'}

In [68]:
conversation = Conversation()
conversation.add_message("user", "Extract company name from bio: " + bio)

In [69]:
#with function call
chat_response = chat_completion_request(
    conversation.conversation_history,
    functions = functions
)
chat_response.json()

{'id': 'chatcmpl-7SLeSBscEXi8O4x8N0fE9giJitvmp',
 'object': 'chat.completion',
 'created': 1686990768,
 'model': 'gpt-4-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'extract_company_name',
     'arguments': '{\n  "companyName": "Devoxx Belgium"\n}'}},
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 252, 'completion_tokens': 20, 'total_tokens': 272}}